In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            #yield i, (j-tf.reduce_mean(j, axis=0))/(tf.math.reduce_std(j, axis=0)+1e-6)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_deep_phys(train).cache(f'{tmp}/deepphys'), to_tf_deep_phys(valid).cache(f'{tmp}/deepphys')
train, valid = to_tf_deep_phys(train), to_tf_deep_phys(valid)

In [2]:
deepphys = DeepPhys_end_to_end()
deepphys.compile(optimizer='Nadam', loss='mse')
deepphys.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(deepphys, input_sig=[tf.TensorSpec([160, 36, 36, 3])])/160:.0f}')
deepphys.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:52158718
Model: "deep_phys_end_to_end"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [ ]:
deepphys.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/DeepPhys_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
deepphys.load_weights('../weights/DeepPhys_CCNU.h5')

In [3]:
deepphys.load_weights('../weights/DeepPhys_CCNU.h5')

In [ ]:
deepphys(np.random.random((160, 36, 36, 3)))

In [5]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [15]:
eval_on_dataset(test_set_CCNU, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_CCNU.h5')
get_metrics('../results/DeepPhys_CCNU_CCNU.h5')

{'Sliding window': {'MAE': 1.521, 'RMSE': 4.401, 'R': 0.9062},
 'Whole video': {'MAE': 0.951, 'RMSE': 2.926, 'R': 0.95207}}

In [31]:
#eval_on_dataset(test_set_CCNU_rPPG, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
#cumsum('../results/DeepPhys_CCNU_CCNU_rPPG.h5')
#get_metrics('../results/DeepPhys_CCNU_CCNU_rPPG.h5')
get_metrics_HRV('../results/DeepPhys_CCNU_CCNU_rPPG.h5')

{'SDNN': {'MAE': 57.574, 'RMSE': 64.214, 'R': 0.33789}}

In [28]:
eval_on_dataset(test_set_PURE, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_PURE.h5')
#get_metrics('../results/DeepPhys_CCNU_PURE.h5')
get_metrics_HRV('../results/DeepPhys_CCNU_PURE.h5')

100%|██████████| 59/59 [00:33<00:00,  1.75it/s]
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\scipy\interpolate\_fitpack2.py:298: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
x:\Users\wkg\a

{'SDNN': {'MAE': 85.98, 'RMSE': 92.018, 'R': 0.29681}}

In [30]:
eval_on_dataset(test_set_UBFC_rPPG2, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_UBFC.h5')
#get_metrics('../results/DeepPhys_CCNU_UBFC.h5')
get_metrics_HRV('../results/DeepPhys_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:22<00:00,  1.88it/s]


{'SDNN': {'MAE': 30.012, 'RMSE': 37.764, 'R': 0.64769}}

In [5]:
eval_on_dataset(test_set_MMPD, deepphys, 160, (36, 36), step=4, batch=0, save='../results/DeepPhys_CCNU_MMPD.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/DeepPhys_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'LED-low', 'Incandescent'])

100%|██████████| 660/660 [02:42<00:00,  4.05it/s]


{'Sliding window': {'MAE': 2.046, 'RMSE': 6.166, 'R': 0.77657},
 'Whole video': {'MAE': 1.026, 'RMSE': 1.455, 'R': 0.98737}}

In [5]:
eval_on_dataset(test_set_COHFACE, deepphys, 160, (36, 36), step=4, batch=0, save='../results/DeepPhys_CCNU_COHFACE.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/DeepPhys_CCNU_COHFACE.h5')

100%|██████████| 164/164 [00:47<00:00,  3.43it/s]


{'Sliding window': {'MAE': 4.028, 'RMSE': 10.57, 'R': 0.62154},
 'Whole video': {'MAE': 2.753, 'RMSE': 8.627, 'R': 0.73356}}